In [1]:
from keras.applications.resnet50 import ResNet50
import numpy as np
np.set_printoptions(threshold=np.nan)

from art.classifiers import KerasClassifier
import keras.backend as k

k.set_learning_phase(0)

# Load model
mean_imagenet = np.zeros([224, 224,3])
mean_imagenet[...,0].fill(103.939)
mean_imagenet[...,1].fill(116.779)
mean_imagenet[...,2].fill(123.68)
model = ResNet50(weights='imagenet')
classifier = KerasClassifier(model=model, clip_values=(0, 255),  preprocessing=(mean_imagenet,1))

Using TensorFlow backend.


Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [2]:
from os.path import join, abspath, expanduser

# Get Imagenet labels
path = expanduser('~/git/nemesis/')
with open(join(path, "imagenet/labels.txt"), "r") as f_input:
    class_names = eval(f_input.read())

In [3]:
from keras.preprocessing import image

path = expanduser('~/git/nemesis/demo-pics')
image_files = ['A.jpg', 'B.jpg', 'C.jpg', 'D.jpg', 'E.jpg']
fnames = ['A', 'B', 'C', 'D', 'E']
images = []

for i in image_files:
    image_file = join(path, i)
    image_ = image.load_img(image_file, target_size=(224, 224))
    img = image.img_to_array(image_)
    images.append(img)
images = np.array(images)
    
# Original predictions
preds = classifier.predict(images)
labels = np.argmax(preds, axis=1)
conf = np.max(preds, axis=1)
print('Predictions:')
for i in range(len(preds)):
    print(class_names[labels[i]], conf[i])

Predictions:
mousetrap 0.9998363
car wheel 0.95073944
spotlight, spot 0.43749318
Siamese cat, Siamese 0.8781743
cup 0.7446252


In [4]:
levels = ['low', 'med', 'high']

In [5]:
# Create attacks
from art.attacks import FastGradientMethod, SaliencyMapMethod, CarliniL2Method
from art.attacks import ProjectedGradientDescent, BasicIterativeMethod

attacks_config = {
    'fgsm': {'class': FastGradientMethod,
             'low': {'eps': 5},
             'med': {'eps': 10},
             'high': {'eps': 20}},
    'carlini': {'class': CarliniL2Method,
               'low': {'confidence': 0, 'targeted': False, 'binary_search_steps': 10, 'max_iter': 10},
               'med': {'confidence': 20, 'targeted': False, 'binary_search_steps': 20, 'max_iter': 20},
               'high': {'confidence': 50, 'targeted': False, 'binary_search_steps': 20, 'max_iter': 20}},
    'jsma': {'class': SaliencyMapMethod,
             'low': {'theta': 1, 'gamma': .1},
             'med': {'theta': 5, 'gamma': .3},
             'high': {'theta': 10, 'gamma': .5}},
    'bim':  {'class': BasicIterativeMethod,
             'low': {'eps': 8, 'eps_step': 1, 'max_iter': 5},
             'med': {'eps': 8, 'eps_step': .5, 'max_iter': 50},
             'high': {'eps': 8, 'eps_step': .1, 'max_iter': 100}},
    'pgd':  {'class': ProjectedGradientDescent,
             'low': {'eps': 8, 'eps_step': 1, 'max_iter': 5},
             'med': {'eps': 8, 'eps_step': .5, 'max_iter': 50},
             'high': {'eps': 8, 'eps_step': .1, 'max_iter': 100}},
}

In [6]:
def get_attack(classifier, name, level='low'):
    try:
        attacker = attacks_config[name]['class'](classifier)
        attacker.set_params(**attacks_config[name][level])
        return attacker
    except:
        raise KeyError()

In [7]:
# Create defenses
from art.defences import GaussianAugmentation, FeatureSqueezing, SpatialSmoothing

defenses_config = {
    'ga': {'class': GaussianAugmentation,
          'low': {'sigma': 3, 'augmentation': False},
          'med': {'sigma': 6, 'augmentation': False},
          'high': {'sigma': 9, 'augmentation': False}},
    'fs': {'class': FeatureSqueezing,
          'low': {'bit_depth': 7},
          'med': {'bit_depth': 5},
          'high': {'bit_depth': 3}},
    'ss': {'class': SpatialSmoothing,
          'low': {'window_size': 3},
          'med': {'window_size': 9},
          'high': {'window_size': 15}}
}

In [8]:
def get_defense(name, level='low'):
    try:
        defense = defenses_config[name]['class'](**defenses_config[name][level])
        return defense
    except:
        raise KeyError()

In [9]:
from PIL import Image

# Compute and save all images
attacks = ['bim', 'fgsm', 'pgd']
# levels = ['low']

levels_def = [None, 'low', 'med', 'high']

for a in attacks:
    # Run attack
    for la in levels:
        print('-- %s %s' % (a, la))
        attack = get_attack(classifier, a, level=la)

        images_adv = attack.generate(images)
        images_adv = np.round(images_adv).astype('uint8')
#         preds = classifier.predict(images_adv)
#         labels = np.argmax(preds, axis=1)
#         conf = np.max(preds, axis=1)
#         print('Predictions:')
#         for i, image in enumerate(images_adv):
#             print(class_names[labels[i]], conf[i])
            
#             # Save attack images here
#             im = Image.fromarray(image)
#             im.show()
#             im.save('%s-%s-%s-None-None-None-%s-%i.jpg' % (fnames[i], a, la, class_names[labels[i]].split(',')[0].replace(' ', '_'),
#                                             np.round(conf[i] * 100)))

        # Now add defenses to the attack
        for level_ss in levels_def:
            if level_ss is not None:
                ss = get_defense('ss', level=level_ss)
                images_ss = ss(images_adv)
            else:
                images_ss = images_adv.copy()

            for level_fs in levels_def:
                if level_fs is not None:            
                    fs = get_defense('fs', level=level_fs)
                    images_fs = fs(images_ss, clip_values=(0, 255))
                else:
                    images_fs = images_ss.copy()

                for level_ga in levels_def:
                    print('-- %s %s %s' % (level_ss, level_fs, level_ga))
                    if level_ga is not None:
                        ga = get_defense('ga', level=level_ga)
                        images_ga = ga(images_fs)
                    else:
                        images_ga = images_fs.copy()
                    images_ga = np.round(images_ga).astype('uint8')

                    preds = classifier.predict(images_ga)
                    labels = np.argmax(preds, axis=1)
                    conf = np.max(preds, axis=1)
                    print('Predictions:')

                    # Save defense images here
                    for i, image in enumerate(images_ga):
                        print(class_names[labels[i]], conf[i])
                        im = Image.fromarray(image)
                        im.save('%s-%s-%s-%s-%s-%s-%s-%i.jpg' % (fnames[i], a, la,
                                                                     str(level_ss), str(level_fs), str(level_ga),
                                                                     class_names[labels[i]].split(',')[0].replace(' ', '_'),
                                                                     np.round(conf[i] * 100)))

# Order for applying defenses: SS, FS, GA
# Now compute with defenses alone
# for level_ss in levels:
#     if level_ss is not None:
#         ss = get_defense('ss', level=level_ss)
#         images_ss = ss(images)
#     else:
#         images_ss = images.copy()
        
#     for level_fs in levels:
#         if level_fs is not None:            
#             fs = get_defense('fs', level=level_fs)
#             images_fs = fs(images_ss, clip_values=(0, 255))
#         else:
#             images_fs = images_ss.copy()
#         for level_ga in levels:
#             print('-- %s %s %s' % (level_ss, level_fs, level_ga))
#             if level_ga is not None:
#                 ga = get_defense('ga', level=level_ga)
#                 images_ga = ga(images_fs)
#             else:
#                 images_ga = images_fs.copy()
#             images_ga = np.round(images_ga).astype('uint8')

#             preds = classifier.predict(images_ga)
#             labels = np.argmax(preds, axis=1)
#             conf = np.max(preds, axis=1)
#             print('Predictions:')

#             # Save defense images here
#             for i, image in enumerate(images_ga):
#                 print(class_names[labels[i]], conf[i])
#                 im = Image.fromarray(image)
#                 im.save('%s-None-None-%s-%s-%s-%s-%i.jpg' % (fnames[i], str(level_ss), str(level_fs), str(level_ga),
#                                                              class_names[labels[i]].split(',')[0].replace(' ', '_'),
#                                                              np.round(conf[i] * 100)))

-- bim low
-- None None None
Predictions:
carton 0.20492321
disk brake, disc brake 0.5121055
screwdriver 0.6560746
altar 0.31307697
caldron, cauldron 0.45226517
-- None None low
Predictions:
carton 0.109503254
disk brake, disc brake 0.4745864
syringe 0.6271833
sewing machine 0.4093424
caldron, cauldron 0.77939165
-- None None med
Predictions:
envelope 0.13839476
car wheel 0.40459606
screwdriver 0.6153819
sewing machine 0.2512243
caldron, cauldron 0.6028585
-- None None high
Predictions:
envelope 0.23743261
car wheel 0.43999946
screwdriver 0.4029548
altar 0.12534474
caldron, cauldron 0.4958641
-- None low None
Predictions:
carton 0.14779596
disk brake, disc brake 0.545829
screwdriver 0.54221123
altar 0.4158045
caldron, cauldron 0.4822619
-- None low low
Predictions:
carton 0.1450373
disk brake, disc brake 0.47787777
syringe 0.5149308
altar 0.41566485
caldron, cauldron 0.7368576
-- None low med
Predictions:
envelope 0.20094341
car wheel 0.4594089
syringe 0.29828635
altar 0.32372546
caldr

Predictions:
volcano 0.26605785
strainer 0.15206523
matchstick 0.48211783
traffic light, traffic signal, stoplight 0.27213725
stove 0.13864146
-- high med None
Predictions:
pinwheel 0.63283193
bubble 0.31351086
web site, website, internet site, site 0.12342163
toyshop 0.1480108
candle, taper, wax light 0.43486688
-- high med low
Predictions:
volcano 0.27530545
torch 0.86040306
lemon 0.19606528
torch 0.14718248
stove 0.10493986
-- high med med
Predictions:
goldfish, Carassius auratus 0.33691013
torch 0.20214094
lighter, light, igniter, ignitor 0.36293602
pillow 0.06295836
stove 0.3295153
-- high med high
Predictions:
traffic light, traffic signal, stoplight 0.4435177
strainer 0.19844022
lighter, light, igniter, ignitor 0.5563513
goldfish, Carassius auratus 0.21310653
stove 0.29337212
-- high high None
Predictions:
matchstick 0.10401648
torch 0.26422876
web site, website, internet site, site 0.38076264
digital clock 0.53132504
pug, pug-dog 0.10432687
-- high high low
Predictions:
envelop

Predictions:
goldfish, Carassius auratus 0.32227007
candle, taper, wax light 0.6543874
matchstick 0.36361605
goldfish, Carassius auratus 0.10575555
stove 0.8066103
-- med high None
Predictions:
goldfish, Carassius auratus 0.33267903
torch 0.5716872
spatula 0.10196629
toyshop 0.08890999
candle, taper, wax light 0.32735562
-- med high low
Predictions:
goldfish, Carassius auratus 0.25910434
torch 0.51676923
spatula 0.17722487
ice cream, icecream 0.12427395
comic book 0.06487165
-- med high med
Predictions:
goldfish, Carassius auratus 0.1894548
candle, taper, wax light 0.3926684
matchstick 0.4670355
traffic light, traffic signal, stoplight 0.13111117
candle, taper, wax light 0.06802282
-- med high high
Predictions:
goldfish, Carassius auratus 0.28562656
ice cream, icecream 0.15155742
matchstick 0.14847435
digital clock 0.09322773
microwave, microwave oven 0.08810939
-- high None None
Predictions:
pinwheel 0.6968334
torch 0.97208595
web site, website, internet site, site 0.1641119
pinwheel 

Predictions:
pinwheel 0.5794033
torch 0.6619124
hook, claw 0.3330517
torch 0.3284652
mask 0.15292893
-- med None low
Predictions:
pinwheel 0.32953602
candle, taper, wax light 0.4939167
ice lolly, lolly, lollipop, popsicle 0.10679639
bubble 0.078937985
matchstick 0.102639064
-- med None med
Predictions:
traffic light, traffic signal, stoplight 0.2037131
candle, taper, wax light 0.37314737
lighter, light, igniter, ignitor 0.48084375
pillow 0.07816816
stove 0.25419343
-- med None high
Predictions:
traffic light, traffic signal, stoplight 0.19130136
candle, taper, wax light 0.17103752
matchstick 0.382634
tray 0.18561752
stove 0.66260505
-- med low None
Predictions:
pinwheel 0.3720217
torch 0.8438259
hook, claw 0.33043882
torch 0.4800748
mask 0.1853404
-- med low low
Predictions:
pinwheel 0.65624964
candle, taper, wax light 0.8040222
hook, claw 0.09129164
bubble 0.10966343
stove 0.08018214
-- med low med
Predictions:
goldfish, Carassius auratus 0.14625442
candle, taper, wax light 0.4351846


Predictions:
carton 0.06374414
torch 0.40985316
syringe 0.44758028
altar 0.27159774
candle, taper, wax light 0.47296247
-- low low low
Predictions:
space shuttle 0.07470175
disk brake, disc brake 0.51833975
ice lolly, lolly, lollipop, popsicle 0.25774482
altar 0.3471628
cup 0.57788336
-- low low med
Predictions:
envelope 0.17446694
disk brake, disc brake 0.40778106
screwdriver 0.27639025
altar 0.18133238
cup 0.640293
-- low low high
Predictions:
envelope 0.18904747
disk brake, disc brake 0.47105464
syringe 0.17318642
altar 0.31863627
cup 0.3529762
-- low med None
Predictions:
space shuttle 0.07198924
car wheel 0.57424307
syringe 0.6346476
altar 0.4394661
cup 0.7440094
-- low med low
Predictions:
carton 0.12581849
disk brake, disc brake 0.42786005
syringe 0.7234815
altar 0.39391914
cup 0.6950409
-- low med med
Predictions:
wing 0.09370706
disk brake, disc brake 0.5400075
screwdriver 0.2976909
altar 0.24120466
cup 0.5911762
-- low med high
Predictions:
envelope 0.23903859
disk brake, dis

...

Predictions:
goldfish, Carassius auratus 0.16818199
candle, taper, wax light 0.6374928
lighter, light, igniter, ignitor 0.21705294
goldfish, Carassius auratus 0.15400088
candle, taper, wax light 0.33263803
-- med high med
Predictions:
tray 0.32863575
candle, taper, wax light 0.89139384
lighter, light, igniter, ignitor 0.49411908
traffic light, traffic signal, stoplight 0.19299091
jigsaw puzzle 0.29072663
-- med high high
Predictions:
tray 0.29904848
candle, taper, wax light 0.54901206
lighter, light, igniter, ignitor 0.7373457
traffic light, traffic signal, stoplight 0.3469549
fountain 0.2229119
-- high None None
Predictions:
torch 0.35047868
torch 0.8555014
traffic light, traffic signal, stoplight 0.25288635
sea slug, nudibranch 0.27397627
candle, taper, wax light 0.39065698
-- high None low
Predictions:
screen, CRT screen 0.16975681
torch 0.59694725
lighter, light, igniter, ignitor 0.629238
torch 0.23278366
bubble 0.111570105
-- high None med
Predictions:
window screen 0.43688205
spi

Predictions:
goldfish, Carassius auratus 0.37254694
torch 0.80645853
matchstick 0.5468591
sewing machine 0.0898565
stove 0.30386394
-- med low None
Predictions:
torch 0.40912724
torch 0.92531055
hook, claw 0.42383325
torch 0.4422567
candle, taper, wax light 0.28694457
-- med low low
Predictions:
pinwheel 0.8189208
torch 0.7544974
matchstick 0.11885442
pillow 0.08150838
cup 0.13596767
-- med low med
Predictions:
goldfish, Carassius auratus 0.40775508
ice cream, icecream 0.24597447
lighter, light, igniter, ignitor 0.37566826
pillow 0.08890417
caldron, cauldron 0.3744098
-- med low high
Predictions:
goldfish, Carassius auratus 0.598248
torch 0.3319338
lighter, light, igniter, ignitor 0.69607234
goldfish, Carassius auratus 0.12665118
caldron, cauldron 0.47986513
-- med med None
Predictions:
torch 0.6403154
candle, taper, wax light 0.96827847
hook, claw 0.090753436
toyshop 0.11886355
cup 0.08227587
-- med med low
Predictions:
pinwheel 0.18207707
torch 0.639222
ice lolly, lolly, lollipop, po

Predictions:
envelope 0.29411915
disk brake, disc brake 0.3982343
matchstick 0.30766544
altar 0.5835269
caldron, cauldron 0.7102958
-- low low high
Predictions:
envelope 0.4530706
disk brake, disc brake 0.5088863
spotlight, spot 0.13820241
altar 0.14001611
cup 0.50495476
-- low med None
Predictions:
space shuttle 0.2003591
car wheel 0.504045
ice lolly, lolly, lollipop, popsicle 0.47342178
altar 0.65977293
candle, taper, wax light 0.7206465
-- low med low
Predictions:
carton 0.15707679
disk brake, disc brake 0.53619313
ice lolly, lolly, lollipop, popsicle 0.6718482
altar 0.5531022
caldron, cauldron 0.3699456
-- low med med
Predictions:
envelope 0.16947076
disk brake, disc brake 0.52933186
syringe 0.7216214
altar 0.6427645
caldron, cauldron 0.72778976
-- low med high
Predictions:
envelope 0.32954943
disk brake, disc brake 0.4751101
matchstick 0.42510018
altar 0.12484691
cup 0.5488861
-- low high None
Predictions:
envelope 0.21332084
car wheel 0.82210594
syringe 0.36811864
altar 0.4169024

Predictions:
envelope 0.22788319
disk brake, disc brake 0.6206888
syringe 0.5861737
altar 0.6508077
caldron, cauldron 0.7438135
-- None med med
Predictions:
envelope 0.41900995
disk brake, disc brake 0.52630424
screwdriver 0.8617176
altar 0.12933755
caldron, cauldron 0.7340458
-- None med high
Predictions:
envelope 0.21697521
car wheel 0.49474975
screwdriver 0.64582
altar 0.18079281
cup 0.6643991
-- None high None
Predictions:
envelope 0.3469914
car wheel 0.46133423
syringe 0.42953402
altar 0.18934403
cup 0.71256405
-- None high low
Predictions:
envelope 0.34823003
car wheel 0.4709063
screwdriver 0.5566654
altar 0.21750492
cup 0.6895011
-- None high med
Predictions:
envelope 0.24437153
car wheel 0.4271743
screwdriver 0.71691954
traffic light, traffic signal, stoplight 0.19240098
cup 0.8780282
-- None high high
Predictions:
menu 0.17393138
car wheel 0.48743907
screwdriver 0.23407543
altar 0.19907723
cup 0.69686645
-- low None None
Predictions:
envelope 0.23001759
torch 0.347416
syringe 

Predictions:
envelope 0.27468812
stove 0.16654184
lighter, light, igniter, ignitor 0.21977408
digital clock 0.14423491
jigsaw puzzle 0.098126456
-- high high high
Predictions:
goldfish, Carassius auratus 0.39328635
stove 0.44725394
lighter, light, igniter, ignitor 0.31592116
digital clock 0.18889664
traffic light, traffic signal, stoplight 0.0962387
